# Extract the Data

In [1]:
import psycopg2
import re
import requests
from ollama import Client
import data_connector as dc

# Processing jobs for each platform
# 10 = tiktok
# 20 = youtube
# 30 = instagram_post
# 40 = facebook_post
# 50 = google_result
# 60 = twitter_tweets

# Step 1: Extract Data
def fetch_posts(social_media_monitoring_id, platform_id):
    conn = psycopg2.connect(
        dbname='medols',
        user='postgres',
        password='FEWcTB3JIX5gK4T06c1MdkM9N2S8w9pb',
        host='98.98.117.105',
        port='5432')

    if (platform_id == 60):
        ## twitter
        query = """
        select tweet as post_content 
        from twitter_tweets 
        where monitoring_id = '%s'
        """ % social_media_monitoring_id
    elif (platform_id == 50):
        ## google result
        """ % social_media_monitoring_id
    elif (platform_id == 40):
        ## facebook post
        query = """
        select tweet as post_content 
        from twitter_tweets 
        where monitoring_id = '%s'
        """ % social_media_monitoring_id
    elif (platform_id == 30):
        ## instagram post
        query = """
        select tweet as post_content 
        from twitter_tweets 
        where monitoring_id = '%s'
        """ % social_media_monitoring_id
    elif (platform_id == 20):
        ## Youtube Post
        query = """
        select tweet as post_content 
        from twitter_tweets 
        where monitoring_id = '%s'
        """ % social_media_monitoring_id
    elif (platform_id == 10):
        ## tiktok post
        query = """
        select tweet as post_content 
        from twitter_tweets 
        where monitoring_id = '%s'
        """ % social_media_monitoring_id    
    
    cursor = conn.cursor()
    cursor.execute(query)
    posts = cursor.fetchall()
    cursor.close()
    conn.close()
    return [post[0] for post in posts]


# Pre Process the Data

In [2]:
# Step 2: Preprocess Data
def preprocess_text(text):
    # Remove URLs
    text = re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE)
    # Remove user mentions
    text = re.sub(r'@\w+', '', text)
    # Remove hashtags
    text = re.sub(r'#\w+', '', text)
    # Remove special characters and numbers
    text = re.sub(r'\W+|\d+', ' ', text)
    # Convert to lowercase
    text = text.lower()
    return text

# Query to LLM

In [3]:
# Step 3: Query the Hosted LLM
def query_ollama_llm(texts, num_topics=5):
    client = Client(host='http://localhost:11434')

    prompt_txt = """
Anda adalah seorang Ahli Analitik Media Sosial Lanjutan. Berdasarkan postingan Twitter di bawah ini, tentukan 5 topik utama yang mencakup keseluruhan. Sertakan tema-tema kunci, kata kunci, dan penjelasan singkat untuk setiap topik yang diidentifikasi:
Di bawah ini adalah postingan Twitter:

{post1}
{post2}
{post3}
...
N. {postN}
    """

    prompt_txt = prompt_txt + '\n\n'
    prompt_txt = prompt_txt.join(texts)
    
    prompt_msg = [
      {
        'role': 'user',
        'content': prompt_txt,
      },
    ]
    # print(prompt_msg[:1000])
    
    response = client.chat(model='llama3', messages=prompt_msg)
    # return ""
    return response



# Step 4: Recording to database

In [4]:
def record_to_database(topics_output, social_media_monitoring_id, platform_id):

    # Establish connection
    conn = psycopg2.connect(
        dbname='medols',
        user='postgres',
        password='FEWcTB3JIX5gK4T06c1MdkM9N2S8w9pb',
        host='98.98.117.105',
        port='5432'
    )


    
    # Create a cursor
    cur = conn.cursor()
    
    # Text containing apostrophe and double quotes
    # text = 'It\'s a "beautiful" day.'
    
    #preparing all the data component
    model = topics_output['model']
    created_at = topics_output['created_at']
    total_duration = topics_output['total_duration']
    load_duration = topics_output['load_duration']
    prompt_eval_duration = topics_output['prompt_eval_duration']
    eval_count = topics_output['eval_count']
    eval_duration = topics_output['eval_duration']
    role = topics_output['message']['role']
    text_output = "$$" + topics_output['message']['content'] + "$$"

    ssql = """
    insert into topic_detection (social_media_monitoring_id, model, created_at ,total_duration, 
					 	load_duration, prompt_eval_duration, eval_count, eval_duration,
					 	llm_role, topic_detection, platform_id)
    values ('%s','%s','%s',%s ,%s ,%s ,%s ,%s, '%s', %s, %s)
    """
    ssql = ssql % (social_media_monitoring_id, model, created_at, total_duration, load_duration, prompt_eval_duration, eval_count, eval_duration, role, text_output, platform_id)
    print(ssql)
    # dc.execute_query_psql(ssql)

    # Parameterized query
    cur.execute(ssql, (social_media_monitoring_id, model, created_at, total_duration, load_duration, prompt_eval_duration, eval_count, eval_duration, role, text_output, platform_id,))
    
    # Commit the transaction
    conn.commit()
    
    # Close the cursor and connection
    cur.close()
    conn.close()

def record_calculated(social_media_monitoring_id):
    ssql = "insert into topic_detection_rec (monitoring_search_id, process_time) values ('%s', NOW())"
    ssql = ssql % (social_media_monitoring_id)
    dc.execute_query_psql(ssql)
    
    

In [5]:
import json
import textwrap
import logging

# Set up logging to capture errors
logging.basicConfig(level=logging.INFO)

social_media_monitoring_id = ""
platform_id = 0

# query for outstanding
ssql = """
select 	b.*
from 	monitoring_search a inner join monitoring_media_social b 
		on a.id = b.monitoring_search_id 
where 	b.id not in (select monitoring_search_id from topic_detection_rec)
        and a.status = 3
order by a.created_date desc 
limit 1
"""
df_source = dc.execute_query_psql(ssql)
df_source.head()

social_media_monitoring_id = df_source.iloc[0]['id']
print(social_media_monitoring_id)

# Processing jobs for each platform
# 10 = tiktok
# 20 = youtube
# 30 = instagram_post
# 40 = facebook_post
# 50 = google_result
# 60 = twitter_tweets

if (df_source.iloc[0]['is_twitter']):
    print('process twitter')
    platform_id = 60
    
    try:
        # Fetch the posts
        posts = fetch_posts(social_media_monitoring_id, platform_id)
        
        # Preprocess the posts
        cleaned_posts = [preprocess_text(post) for post in posts]
        
        # topics = query_ollama_llm(cleaned_posts, num_topics=5)
        topics = query_ollama_llm(cleaned_posts,5)
        
        # recording to database
        record_to_database(topics, social_media_monitoring_id, platform_id)
    except Exception as e:
        logging.error(f"Error fetching posts: {e}")
        posts = []



54832fdd-0c68-4360-9858-4a56966f8b97
process twitter


INFO:httpx:HTTP Request: POST http://localhost:11434/api/chat "HTTP/1.1 200 OK"



    insert into topic_detection (social_media_monitoring_id, model, created_at ,total_duration, 
					 	load_duration, prompt_eval_duration, eval_count, eval_duration,
					 	llm_role, topic_detection, platform_id)
    values ('54832fdd-0c68-4360-9858-4a56966f8b97','llama3','2024-08-22T04:03:26.116749046Z',334434017090 ,36993452628 ,11730888000 ,407 ,285252759000, 'assistant', $$Based on the Twitter posts, I identified 5 main topics that cover the entire issue:

**Topic 1: Infidelity (Selingkuh)**
Tema kunci: Selingkuh, Azizah Salsha, Pratama Arhan
Penjelasan singkat: The main topic is about the infidelity of Azizah Salsha, who was caught having an affair with another man named Salim Nauderer.

**Topic 2: Scandal (Kasus)**
Tema kunci: Kasus, Perselingkuhan, Azizah Salsha
Penjelasan singkat: The scandal surrounding Azizah Salsha's infidelity has gone viral, causing a stir on social media and sparking public interest.

**Topic 3: Clarification (Klarifikasi)**
Tema kunci: Klarifikasi, Sa

In [6]:
# record as calculated
record_calculated(social_media_monitoring_id)